In [230]:
import pandas as pd

In [231]:
df=pd.read_excel('Decathlon Apparel Data.xlsx')

In [232]:
df

,product_name,brand,star_rating,number_of_reviews,MRP(in rs),MRP,sale_price(in rs),sale_price,colour,description,Quantity,amount,transaction_Id,transaction date,User_ID,Cust_name,Gender,Age
0,100 Base Layer Ski Pants Women's,Wedze,4.4,3101,836.2629,9.99,836.2629,9.99,Black,"[""100 Base Layer Ski Pants Women's | Designed ...",3,2508.7887,156752,2022-05-01,1002903,Sanskriti,F,28
1,Thermal Underwear Base Layer Top Women's,Wedze,4.5,7367,836.2629,9.99,669.6800,8.00,Black,"[""Thermal Underwear Base Layer Top Women's | D...",2,1339.3600,156753,2022-05-02,1000732,Kartik,F,35
2,100 Ski Jacket Men's,Wedze,4.6,3798,3347.5629,39.99,2511.3000,30.00,Black,"[""100 Ski Jacket Men's | This warm and comfort...",1,2511.3000,156754,2022-05-03,1001990,Bindu,F,35
3,Forclaz Trek 500 Merino Wool Hiking Hat Adult,Forclaz,4.6,1803,2091.9129,24.99,2091.9129,24.99,Whale Gray,['Forclaz Trek 500 Merino Wool Hiking Hat Adul...,1,2091.9129,156755,2022-05-04,1001425,Sudevi,M,16
4,Wedze FR100 Freeride Ski Jacket Women's,Wedze,4.5,239,9961.4900,119.00,5859.7000,70.00,Light Gray,"[""Wedze FR100 Freeride Ski Jacket Women's | Ou...",3,17579.1000,156756,2022-05-05,1000588,Joni,M,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
634,"MH K100, Category 3 Hiking Sunglasses, Kids'",Quechua,4.6,1307,1087.3929,12.99,669.6800,8.00,Blush Pink,"[""MH K100, Category 3 Hiking Sunglasses, Kids'...",2,1339.3600,157386,2024-01-25,1004682,Vishakha,F,27
635,Women's Cardio Training Fitness Shoes Mid 120,Domyos,4.4,1691,2929.0129,34.99,2092.7500,25.00,Black,"[""Women's Cardio Training Fitness Shoes Mid 12...",1,2092.7500,157387,2024-01-26,1004791,Crebagga,F,19
636,Keepdry 100 Long-Sleeved Soccer Base Layer Top ',Kipsta,4.7,1483,1087.3929,12.99,418.5500,5.00,Bright Indigo,"[""Keepdry 100 Long-Sleeved Soccer Base Layer T...",3,1255.6500,157388,2024-01-27,1002181,Sweta,F,32
637,Adult Football Bottoms Essential - Black,Kipsta,4.6,705,2091.9129,24.99,1255.6500,15.00,Black,['Adult Football Bottoms Essential - Black | O...,2,2511.3000,157389,2024-01-28,1002109,Gelder,F,34


## dropping unnecessary columns

In [233]:
df=df.drop(columns=["number_of_reviews","MRP","sale_price","colour","description"])

In [234]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 639 entries, 0 to 638
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   product_name       639 non-null    object        
 1   brand              639 non-null    object        
 2   star_rating        639 non-null    float64       
 3   MRP(in rs)         639 non-null    float64       
 4   sale_price(in rs)  639 non-null    float64       
 5   Quantity           639 non-null    int64         
 6   amount             639 non-null    float64       
 7   transaction_Id     639 non-null    int64         
 8   transaction date   639 non-null    datetime64[ns]
 9   User_ID            639 non-null    int64         
 10  Cust_name          639 non-null    object        
 11  Gender             639 non-null    object        
 12  Age                639 non-null    int64         
dtypes: datetime64[ns](1), float64(4), int64(4), object(4)
memory usag

## feature engineering

We need to create columns for checking the Recency (time since last purchase), Frequency (number of purchases) and Monetary (total amount spent) value of customers.

In [235]:
#convert transaction date to date time format
df["transaction date"]=pd.to_datetime(df["transaction date"])

In [236]:
#Defining reference date for calculating recency
reference_date = df['transaction date'].max()
reference_date

Timestamp('2024-01-29 00:00:00')

In [242]:
# Recency: Calculate the number of days since the customer's last purchase
recency_df = df.groupby('User_ID')['transaction date'].max().reset_index()
recency_df.columns = ['User_ID', 'last_purchase_date']
recency_df['recency'] = (reference_date - recency_df['last_purchase_date']).dt.days

In [215]:
# Frequency: Calculate the total number of transactions per customer
frequency_df = df.groupby('User_ID').size().reset_index()
frequency_df.columns = ['User_ID', 'frequency']

In [216]:
monetary_df = df.groupby('User_ID')['amount'].sum().reset_index()
monetary_df.columns = ['User_ID', 'monetary']

In [247]:
#combining the three dataframes into one 
rfm_df = recency_df.merge(frequency_df, on='User_ID').merge(monetary_df, on='User_ID')

In [218]:
#merging the rfm_df with original dataframe
df = df.merge(rfm_df, on='User_ID', how='left')

In [219]:
df=df.drop(columns=["last_purchase_date"])

In [220]:
df

,product_name,brand,star_rating,MRP(in rs),sale_price(in rs),Quantity,amount,transaction_Id,transaction date,User_ID,Cust_name,Gender,Age,recency,frequency,monetary
0,100 Base Layer Ski Pants Women's,Wedze,4.4,836.2629,836.2629,3,2508.7887,156752,2022-05-01,1002903,Sanskriti,F,28,638,1,2508.7887
1,Thermal Underwear Base Layer Top Women's,Wedze,4.5,836.2629,669.6800,2,1339.3600,156753,2022-05-02,1000732,Kartik,F,35,16,2,5106.3100
2,100 Ski Jacket Men's,Wedze,4.6,3347.5629,2511.3000,1,2511.3000,156754,2022-05-03,1001990,Bindu,F,35,636,1,2511.3000
3,Forclaz Trek 500 Merino Wool Hiking Hat Adult,Forclaz,4.6,2091.9129,2091.9129,1,2091.9129,156755,2022-05-04,1001425,Sudevi,M,16,635,1,2091.9129
4,Wedze FR100 Freeride Ski Jacket Women's,Wedze,4.5,9961.4900,5859.7000,3,17579.1000,156756,2022-05-05,1000588,Joni,M,28,633,2,40178.2887
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
634,"MH K100, Category 3 Hiking Sunglasses, Kids'",Quechua,4.6,1087.3929,669.6800,2,1339.3600,157386,2024-01-25,1004682,Vishakha,F,27,4,2,6360.2858
635,Women's Cardio Training Fitness Shoes Mid 120,Domyos,4.4,2929.0129,2092.7500,1,2092.7500,157387,2024-01-26,1004791,Crebagga,F,19,3,1,2092.7500
636,Keepdry 100 Long-Sleeved Soccer Base Layer Top ',Kipsta,4.7,1087.3929,418.5500,3,1255.6500,157388,2024-01-27,1002181,Sweta,F,32,2,2,2929.8500
637,Adult Football Bottoms Essential - Black,Kipsta,4.6,2091.9129,1255.6500,2,2511.3000,157389,2024-01-28,1002109,Gelder,F,34,1,3,28126.5600


In [222]:
df.to_csv('decathlon_clean.csv',index=False)